In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder \
.master("local[4]") \
.appName("cleaning") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

In [4]:
adult_train_df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("/home/taha/Downloads/adult.data")

In [5]:
adult_test_df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("/home/taha/Downloads/adult.test")

# VERİ BİRLEŞTİRME

In [6]:
adult_whole_df = adult_train_df.union(adult_test_df)

In [7]:
adult_whole_df.limit(5).toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


# Veri Temizligi Basliyor

###### 1-)Tüm stunlara bosluk kontrolu yapilacak

In [8]:
from pyspark.sql.functions import *

In [9]:
adult_whole_df1=adult_whole_df \
.withColumn("workclass",trim(col("workclass"))) \
.withColumn("education",trim(col("education"))) \
.withColumn("marital_status",trim(col("marital_status"))) \
.withColumn("occupation",trim(col("occupation"))) \
.withColumn("relationship",trim(col("relationship"))) \
.withColumn("race",trim(col("race"))) \
.withColumn("sex",trim(col("sex"))) \
.withColumn("native_country",trim(col("native_country"))) \
.withColumn("output",trim(col("output"))) 


In [10]:
adult_whole_df.count()

48842

In [11]:
adult_whole_df1.count()

48842

###### 2-) output icerisindeki '.' temizligi

In [12]:
adult_whole_df2 = adult_whole_df1 \
.withColumn("output",regexp_replace(col("output"),"<=50K.","<=50K")) \
.withColumn("output",regexp_replace(col("output"),">50K.",">50K")) \

In [13]:
adult_whole_df2.groupBy(col("output")) \
.agg({"*":"count"}) \
.toPandas().head()

,output,count(1)
0,<=50K,37155
1,>50K,11687


###### 3-) null kontrolu

In [14]:
sayac_for_null = 1
for sutun in adult_whole_df2.columns:
    if(adult_whole_df2.filter(col(sutun).isNull()).count()>0):
        print(sayac_for_null,".",sutun,"icerisinde null var")
    else:
        print(sayac_for_null, sutun)
    sayac_for_null+=1   

1 age
2 workclass
3 fnlwgt
4 education
5 education_num
6 marital_status
7 occupation
8 relationship
9 race
10 sex
11 capital_gain
12 capital_loss
13 hours_per_week
14 native_country
15 output


###### 4-) soru isareti kontrolleri

In [15]:
sayac_for_soru = 1
for sutun in adult_whole_df2.columns:
    if(adult_whole_df2.filter(col(sutun).contains("?")).count()>0):
        print(sayac_for_soru,".",sutun,"icerisinde soru isareti var")
    else:
        print(sayac_for_soru, sutun)
    sayac_for_soru+=1   

1 age
2 . workclass icerisinde soru isareti var
3 fnlwgt
4 education
5 education_num
6 marital_status
7 . occupation icerisinde soru isareti var
8 relationship
9 race
10 sex
11 capital_gain
12 capital_loss
13 hours_per_week
14 . native_country icerisinde soru isareti var
15 output


In [16]:
# ? bulunan stunların output ile iliskilerini gorelim
adult_whole_df2.select("workclass","occupation","native_country","output") \
.filter(
    col("workclass").contains("?") |
    col("occupation").contains("?") |
    col("native_country").contains("?") 

) \
.groupBy("workclass","occupation","native_country","output").count() \
.orderBy(col("count").desc()) \
.toPandas().head()

,workclass,occupation,native_country,output,count
0,?,?,United-States,<=50K,2284
1,?,?,United-States,>50K,246
2,Private,Other-service,?,<=50K,100
3,Private,Sales,?,<=50K,55
4,Private,Prof-specialty,?,<=50K,51


In [17]:
# soru isareti dagilimi ve hedef degiskene etkisi tesadüfi gorunuyor
# bu yüzden silelim

In [18]:
adult_whole_df3 = adult_whole_df2 \
.filter(~(

    col("workclass").contains("?") |
    col("occupation").contains("?") |
    col("native_country").contains("?") )
    
)

print(adult_whole_df2.count())
print(adult_whole_df3.count())

48842
45222


###### 5-)Zayif sinifların kaldırılması

In [25]:
adult_whole_df4=adult_whole_df3 \
.filter(
   ~(
      col("workclass").contains("never-worked") |
       col("workclass").contains("without-pay") |
       col("occupation").contains("Armed-Forces") |
       col("native_country").contains("Holand-Netherlands")
   
   )

)

print(adult_whole_df3.count())
print(adult_whole_df4.count())

45222
45207


###### 6-)egitim durumuyla ilgili katagorileri birlestirme 

In [23]:
adult_whole_df5=adult_whole_df4.withColumn("education_merged",
when(col("education").isin("1st-4th","5th-6th","7th-8th"),"Elementry-Scholl")
.when(col("education").isin("9th","10th","11th","12th"),"High-Scholl")
.when(col("education").isin("Masters","Doctorate"),"Postgraduate")
 .when(col("education").isin("Bachelors","Some-College"),"Undergraduate")                                          
.otherwise(col("education"))                                           
                                                                           
)
adult_whole_df5.select("education","education_merged")\
.toPandas().head()

,education,education_merged
0,Bachelors,Undergraduate
1,Bachelors,Undergraduate
2,HS-grad,HS-grad
3,11th,High-Scholl
4,Bachelors,Undergraduate


In [26]:
adult_whole_df5.toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output,education_merged
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K,Undergraduate
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K,Undergraduate
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K,HS-grad
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K,High-Scholl
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K,Undergraduate


In [27]:
#diske yazalim
adult_whole_df5 \
.coalesce(1) \
.write \
.mode("overwrite") \
.option("sep",",") \
.option("header","True") \
.csv("/home/taha/Downloads/adult_preprocessed_pyspark")